# initilise the chemical space to search 


Notebook showing how we can define the search space with conditions

In [1]:

import os

import importlib
import pickle
import pymongo
import pandas as pd
import numpy as np
from stk_search.utils import database_utils


First we need to load the precursor dataframe that defines the building blocks used in the chemical space to search

In [2]:
# load the data and the precursors dataset
importlib.reload(database_utils)
df_precursors_path = "data_example/precursor/df_properties.pkl"

df_precursors = pd.read_pickle(df_precursors_path)


Then we can apply some initial filters to the building blocks

In [3]:

def check_mol(mol):
    if mol.GetNumAtoms()>=30:
        return 0
    for atom in mol.GetAtoms():
        if atom.GetAtomicNum() ==14:
            return 0
        if atom.GetAtomicNum() ==32:
            return 0
    return 1
df_precursors['check'] = df_precursors['mol_opt'].apply(check_mol)
df_precursors = df_precursors[df_precursors['check']==1]
df_precursors.drop(columns=['check'], inplace=True)
df_precursors.shape

(307, 10)

In [4]:
# plot some example of precursors
from rdkit.Chem import Draw
from rdkit import Chem
mol_list = []
mol_label = []
df_to_plot = df_precursors.sample(4)
df_to_plot['mol_opt_smiles'] = df_to_plot['mol_opt'].apply(lambda x : Chem.MolFromSmiles(Chem.MolToSmiles(x)))
mol_list.extend(df_to_plot['mol_opt_smiles'].values)
mol_label.extend(df_to_plot['InChIKey'].values)
# Draw molecules
img = Draw.MolsToGridImage(mol_list, molsPerRow=4, subImgSize=(300, 300),legends=[f"{label}" for label in mol_label])  # adjust molsPerRow and subImgSize as needed
img




To define the search space we can set the conditions on the building blocks and the syntax to follow. 

Below we can also use an interactive feature to change the conditions and visualise its impact on the precalculated search space

In [5]:
#%%  generate the search space
from stk_search import SearchSpace
importlib.reload(SearchSpace)

number_of_fragments = 6
target_name = 'target' # specify the target name ( here target refers to the combined property)
aim = 1.0 # specify the target value

SP = SearchSpace.SearchSpace(number_of_fragments=number_of_fragments,df=df_precursors,generation_type='conditional')
print(SP.syntax)
print(SP.conditions_list)

[0, 1, 2, 3, 4, 5]
[[], [], [], [], [], []]


In [6]:
# save the search space
import pickle
save_dir = 'search_space'
id = 'test'
os.makedirs(save_dir, exist_ok=True)
pickle.dump(SP, open(save_dir+f'/search_space_{id}.pkl', 'wb'))
print(' saved to ', save_dir+f'/search_space_{id}.pkl')

 saved to  search_space/search_space_test.pkl


## Add conditions
the syntax to add conditions on the precursors; for each position of the fragments, we can add a condition on the precursors in the form :

['property_name' #condition# value]

The property name need to be in the precursors dataframe


In [7]:
df_precursors

electron affinity (eV)  homo lumo_gap (eV)  ionisation potential (eV)  \
0                    3.0685            0.095166                     9.1140   
1                    2.8466            0.038435                     9.3325   
2                    2.9365            0.072094                     9.2252   
3                    2.5901            0.064465                     8.9842   
4                    2.5596            0.076792                     8.8405   
..                      ...                 ...                        ...   
448                  2.6030            0.130538                     8.4593   
449                  2.7575            0.185717                     8.3305   
450                  2.5843            0.108520                     8.4455   
451                  2.6193            0.101905                     8.4268   
452                  2.7307            0.153670                     8.4750   

     total energy (au)  HOMO-LUMO GAP (eV)    ES1   fosc1  \
0           -56.555859            1.672081  3.787  0.0003   
1           -56.536670            2.258122  3.514  0.0016   
2           -56.533763            2.009171  3.414  0.0003   
3           -48.098467            2.153852  3.364  0.0008   
4           -48.091982            1.967541  3.342  0.0004   
..                 ...                 ...    ...     ...   
448         -55.633449            1.877721  3.366  0.2111   
449         -55.650544            1.674941  3.322  0.2294   
450         -55.632895            1.879434  3.353  0.2088   
451         -55.641560            1.854837  3.358  0.2148   
452         -55.644476            1.820080  3.351  0.2576   

                        InChIKey  \
0    GVYASUPVXDSGHN-UQCOIBPSSA-N   
1    HNRHIGPDSSXEJA-FARCUNLSSA-N   
2    SNMJIXVGAOBXCS-UHFFFAOYSA-N   
3    LWMBLJLODRWDDF-NYYWCZLTSA-N   
4    BJFIUKSBVAJFOJ-WTKPLQERSA-N   
..                           ...   
448  BHOSZUQMOQVQOO-UHFFFAOYSA-N   
449  MTLFYGNDDYBIIU-JYOAFUTRSA-N   
450  VADGQEXKRFNGDZ-UHFFFAOYSA-N   
451  IIXYXVCGBSIBLJ-ONNFQVAWSA-N   
452  ZYHVJHXGAYIDDI-UHFFFAOYSA-N   

                                              mol_opt  Atom_num  
0    <rdkit.Chem.rdchem.Mol object at 0x7d3148c0b090>        22  
1    <rdkit.Chem.rdchem.Mol object at 0x7d3156ae3c20>        22  
2    <rdkit.Chem.rdchem.Mol object at 0x7d3156ae3c70>        22  
3    <rdkit.Chem.rdchem.Mol object at 0x7d3156ae3cc0>        22  
4    <rdkit.Chem.rdchem.Mol object at 0x7d3156ae3d10>        22  
..                                                ...       ...  
448  <rdkit.Chem.rdchem.Mol object at 0x7d30daa4d7c0>        25  
449  <rdkit.Chem.rdchem.Mol object at 0x7d30daa4d810>        25  
450  <rdkit.Chem.rdchem.Mol object at 0x7d30daa4d860>        25  
451  <rdkit.Chem.rdchem.Mol object at 0x7d30daa4d8b0>        25  
452  <rdkit.Chem.rdchem.Mol object at 0x7d30daa4d900>        25  

[307 rows x 10 columns]

In [8]:

SP.conditions_list = [
    ["'ionisation potential (eV)'#<=#6.5"],
    ["'ionisation potential (eV)'#<=#6.5"],
    ["'ionisation potential (eV)'#<=#6.5"],
    ["'ionisation potential (eV)'#<=#6.5"],
    ["'ionisation potential (eV)'#<=#6.5"],
    ["'ionisation potential (eV)'#<=#6.5"],
]


In [9]:
# save the search space
import pickle
save_dir = 'data/input/STK_search_space'
id = 'test_limited'
os.makedirs(save_dir, exist_ok=True)
pickle.dump(SP, open(save_dir+f'/search_space_{id}.pkl', 'wb'))
print(' saved to ', save_dir+f'/search_space_{id}.pkl')

 saved to  data/input/STK_search_space/search_space_test_limited.pkl


In [11]:

print(f"size of the search space is : {SP.get_space_size():.2e}")
SP.df_precursors.head()


size of the search space is : 0.00e+00


electron affinity (eV)  homo lumo_gap (eV)  ionisation potential (eV)  \
0                  3.0685            0.095166                     9.1140   
1                  2.8466            0.038435                     9.3325   
2                  2.9365            0.072094                     9.2252   
3                  2.5901            0.064465                     8.9842   
4                  2.5596            0.076792                     8.8405   

   total energy (au)  HOMO-LUMO GAP (eV)    ES1   fosc1  \
0         -56.555859            1.672081  3.787  0.0003   
1         -56.536670            2.258122  3.514  0.0016   
2         -56.533763            2.009171  3.414  0.0003   
3         -48.098467            2.153852  3.364  0.0008   
4         -48.091982            1.967541  3.342  0.0004   

                      InChIKey  \
0  GVYASUPVXDSGHN-UQCOIBPSSA-N   
1  HNRHIGPDSSXEJA-FARCUNLSSA-N   
2  SNMJIXVGAOBXCS-UHFFFAOYSA-N   
3  LWMBLJLODRWDDF-NYYWCZLTSA-N   
4  BJFIUKSBVAJFOJ-WTKPLQERSA-N   

                                            mol_opt  Atom_num  
0  <rdkit.Chem.rdchem.Mol object at 0x7d3148c0b090>        22  
1  <rdkit.Chem.rdchem.Mol object at 0x7d3156ae3c20>        22  
2  <rdkit.Chem.rdchem.Mol object at 0x7d3156ae3c70>        22  
3  <rdkit.Chem.rdchem.Mol object at 0x7d3156ae3cc0>        22  
4  <rdkit.Chem.rdchem.Mol object at 0x7d3156ae3d10>        22

## Using the prior information and an iteractive tool to set the search space

In [13]:
df_total_path = "data_example/Molecule_database/30K_benchmark_150524.csv"
df_total_new = pd.read_csv(df_total_path)
print(df_total_new.columns)
df_total_new , df_precursors= database_utils.load_data_from_file(df_total_path, df_precursors_path,add_feature_frag=True)
df_total_new['target'] = (
        -np.abs(df_total_new["ES1"] - 3)
        - np.abs(df_total_new["ionisation potential (eV)"] - 5.5)
        + np.log10(df_total_new["fosc1"])
    )

Index(['InChIKey', 'InChIKey_0', 'InChIKey_1', 'InChIKey_2', 'InChIKey_3',
       'InChIKey_4', 'InChIKey_5', 'num_BB', '_id_x', 'Host IP_x',
       'electron affinity (eV)', 'homo lumo_gap (eV)',
       'ionisation potential (eV)', 'total energy (au)', 'cal_folder_x',
       'xtb_cal_folder', '_id_y', 'Excited state energy (eV)',
       'Excited state oscillator strength', 'Host IP_y', 'cal_folder_y', 'ES1',
       'fosc1', 'target', '2d_tani_pca_1', '2d_tani_pca_2', 'PCA1'],
      dtype='object')


In [14]:
#%%  generate the search space
from stk_search import SearchedSpace
importlib.reload(SearchedSpace)

number_of_fragments = 6
target_name = 'target' # specify the target name ( here target refers to the combined property)
aim = 1.0 # specify the target value

SP = SearchedSpace.SearchedSpace(number_of_fragments=number_of_fragments,df=df_precursors,generation_type='conditional')
SP.generate_interactive_condition_V2(df_total=df_total_new,properties_to_plot=["electron affinity (eV)","target"])

In [17]:
import glob
#%% 

search_spaces_dfs = glob.glob('search_space/*pkl')
for id,df_path_SP in enumerate(search_spaces_dfs):
    print(id,df_path_SP)


0 search_space/search_space_properties.pkl
1 search_space/search_space_test.pkl


In [18]:
df_SP = pd.read_pickle(search_spaces_dfs[0])
df_SP.reset_index(drop=True,inplace=True)
df_SP

number of elements              syntax                conditions  \
0                   0  [0, 1, 2, 3, 4, 5]  [[], [], [], [], [], []]   

   Elements in top 5%  number of elements evaluated  
0                   0                             0

In [19]:
# load the saved dataframe with all the tested conditions
# and output it in order to select a few to add to the bactk experiment

import pickle
from datetime import datetime
import os

def save_SP_for_EXP(df:pd.DataFrame,index_list:list,exp_name:str,number_of_fragments):
    time_now = datetime.now().strftime("%Y_%m_%d_%H_%M_%S")
    save_dir = f'data/input/EXP_search_space/{exp_name}_{time_now}/'
    os.makedirs(save_dir,exist_ok=True)
    SP = SearchedSpace.SearchedSpace(number_of_fragments=number_of_fragments,df=df_precursors,generation_type='conditional')
    for id in index_list:
        SP.conditions_list = df['conditions'].values[id]
        SP.syntax = list(df['syntax'].values[id])[:number_of_fragments]
        SP.update()
        print(f"size of the search space is : {SP.space_size:.2e}")
        print(SP.syntax)
        print(SP.conditions_list)
        pickle.dump(SP,file=open(save_dir+f'/search_space_{id}.pkl','wb'))
        print(' saved to ', save_dir+f'/search_space_{id}.pkl')
    return SP, save_dir
experiment_name = '6_mers_filtered_BB'
SP,save_dir= save_SP_for_EXP(df_SP,[0],experiment_name,number_of_fragments)

size of the search space is : 0.00e+00
[0, 1, 2, 3, 4, 5]
[[], [], [], [], [], []]
 saved to  data/input/EXP_search_space/6_mers_filtered_BB_2024_08_23_15_18_15//search_space_0.pkl
